In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.datasets import mnist

Using TensorFlow backend.
D:\Anaconda\envs\tensorflow2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\tensorflow2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\tensorflow2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\tensorflow2.0\lib\site-packages\tensorflow\py

In [2]:
def load_data():  # categorical_crossentropy
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    number = 10000
    x_train = x_train[0:number]
    y_train = y_train[0:number]
    x_train = x_train.reshape(number, 28 * 28)
    x_test = x_test.reshape(x_test.shape[0], 28 * 28)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    # convert class vectors to binary class matrices
    y_train = np_utils.to_categorical(y_train, 10)
    y_test = np_utils.to_categorical(y_test, 10)
    x_train = x_train
    x_test = x_test
    x_test = np.random.normal(x_test)  # 加噪声
    x_train = x_train / 255
    x_test = x_test / 255

    return (x_train, y_train), (x_test, y_test)

In [4]:
if __name__ == '__main__':
    '''
    注意事项如下：
    1、batch_size=100,epochs=20为宜，batch_size过大会导致loss下降曲线过于平滑而卡在local minima、saddle point或plateau处，batch_size过小会导致update次数过多，运算量太大，速度缓慢，但可以带来一定程度的准确率提高
    2、hidden layer数量不要太多，不然可能会发生vanishing gradient(梯度消失)，一般两到三层为宜
    3、如果layer数量太多，则千万不要使用sigmoid等缩减input影响的激活函数，应当选择ReLU、Maxout等近似线性的activation function(layer数量不多也应该选这两个)
    4、每一个hidden layer所包含的neuron数量，五六百为宜
    5、对于分类问题，loss function一定要使用cross entropy(categorical_crossentropy)，而不是mean square error(mse)
    6、优化器optimizer一般选择adam，它综合了RMSProp和Momentum，同时考虑了过去的gradient、现在的gradient，以及上一次的惯性
    7、如果testing data上准确率很低，training data上准确率比较高，可以考虑使用dropout，Keras的使用方式是在每一层hidden layer的后面加上一句model.add(Dropout(0.5))，其中0.5这个参数你自己定；注意，加了dropout之后在training set上的准确率会降低，但是在testing set上的准确率会提高，这是正常的
    8、如果input是图片的pixel，注意对灰度值进行归一化，即除以255，使之处于0～1之间
    9、最后的output最好同时输出在training set和testing set上的准确率，以便于对症下药
    '''
    # load training data and testing data
    (x_train, y_train), (x_test, y_test) = load_data()

    # step 1: define network structure
    model = Sequential()
    
    # 开始叠加NN，两个隐藏层，每层500个neuron
#     model.add(Dense(input_dim=28 * 28, units=500, activation='relu'))
    # sigmoid 换成 relu 识别率更高
    model.add(Dense(input_dim=28 * 28, units=500, activation='sigmoid'))
    # model.add(Dropout(0.5))
    
    model.add(Dense(units=500, activation='relu'))  # Fully connected的layer——用Dense来表示
    # model.add(Dropout(0.5))
    model.add(Dense(units=10, activation='softmax'))

    # set configurations
    model.compile(loss='categorical_crossentropy',  # cross entropy参数就是categorical_crossentropy(Keras里的写法)
                  optimizer='adam', # Keras里面有诸如：SGD(gradient descent)、RMSprop、Adagrad、Adadelta、Adam、Adamax、Nadam之类的寻找最优参数的方法，它们都是gradient descent的方式
                  metrics=['accuracy'])

    # train model
    model.fit(x_train, y_train, batch_size=100, epochs=20)

    # evaluate the model and output the accuracy
    result_train = model.evaluate(x_train, y_train)
    result_test = model.evaluate(x_test, y_test)
    print('Train Acc:', result_train[1])
    print('Test Acc:', result_test[1])

Epoch 1/20
10000/10000 [==============================] - 1s 52us/step - loss: 0.9135 - accuracy: 0.7468
Epoch 2/20
10000/10000 [==============================] - 0s 43us/step - loss: 0.3463 - accuracy: 0.8990
Epoch 3/20
10000/10000 [==============================] - 0s 40us/step - loss: 0.2812 - accuracy: 0.9190
Epoch 4/20
10000/10000 [==============================] - 0s 38us/step - loss: 0.2502 - accuracy: 0.9254
Epoch 5/20
10000/10000 [==============================] - 0s 38us/step - loss: 0.2203 - accuracy: 0.9344
Epoch 6/20
10000/10000 [==============================] - 0s 38us/step - loss: 0.1973 - accuracy: 0.9424
Epoch 7/20
10000/10000 [==============================] - 0s 38us/step - loss: 0.1760 - accuracy: 0.9482
Epoch 8/20
10000/10000 [==============================] - 0s 37us/step - loss: 0.1546 - accuracy: 0.9547
Epoch 9/20
10000/10000 [==============================] - 0s 41us/step - loss: 0.1363 - accuracy: 0.9593
Epoch 10/20
10000/10000 [==============================

In [ ]:
# 评价方式1，已知输出Label
score = model.evaluate(x_test,y_test)
print('Total loss on Testing Set:',score[0])
print('Accuracy of Testing Set:',score[1])

# 评价方式2， 分类的结果
result = model.predict(x_test)

![1](https://gitee.com/Sakura-gh/ML-notes/raw/master/img/keras-step1.png)

![2](https://gitee.com/Sakura-gh/ML-notes/raw/master/img/keras-step2.png)

![2](https://gitee.com/Sakura-gh/ML-notes/raw/master/img/keras-step3.png)

![4](https://gitee.com/Sakura-gh/ML-notes/raw/master/img/matrix-speed.png)

![5](https://gitee.com/Sakura-gh/ML-notes/raw/master/img/save-load-model.png)